### Input: XYZ
- .xyz file must have absorber placed at center position, (0,0,0)

### Output: xyz

In [44]:
import numpy as np
import os

def read_xyz_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        num_atoms = int(lines[0].strip())
        atoms = []
        atom_types = {}
        for i, line in enumerate(lines[2:]):
            parts = line.split()
            atom = parts[0]
            coordinates = list(map(float, parts[1:4]))

            if atom not in atom_types:
                atom_types[atom] = len(atom_types) + 1  # Assign a unique type for each species

            if all([cord == 0 for cord in coordinates]):
                atom_type = 0 # Type 0 for the center atom
            else: 
                atom_type = atom_types[atom] 
            atoms.append((atom, coordinates, atom_type))

    return num_atoms, atoms

def distance(atom1, atom2):
    return np.linalg.norm(np.array(atom1) - np.array(atom2))

def sort_atoms_by_distance(atoms, reference_atom):
    atoms.sort(key=lambda atom: distance(atom[1], reference_atom))
    return atoms

def add_distance_column(atoms, reference_atom):
    for i, atom in enumerate(atoms):
        dist = distance(atom[1], reference_atom)
        atoms[i] = atom + (dist,)
    return atoms

def assign_atom_index(atoms):
    #atoms.sort(key=lambda atom: atom[-1])  # Sort by distance
    for i, atom in enumerate(atoms):
        #print(atom)
        atoms[i] = atom + (i + 1,)  # Assign atom index starting from 1
    return atoms

def write_sorted_xyz_file(sorted_atoms, output_path):
    with open(output_path, 'w') as file:
        for atom in sorted_atoms:
            row = [atom[1][0], atom[1][1], atom[1][2], atom[2], atom[3], atom[4]]
            for num in row:
                formatted_num = "{:<{width}}".format(round(num,6), width=12)
                file.write(formatted_num)
            file.write('\n')

# Get a list of all files in the current directory
file_list = [file for file in os.listdir() if file.endswith(".xyz")]

# Iterate through each XYZ file and process it
for file_name in file_list:
    input_file_path = file_name
    output_file_path = file_name.replace(".xyz", ".txt")

    num_atoms, atoms = read_xyz_file(input_file_path)

    # Assuming the reference atom is at (0, 0, 0)
    reference_atom = (0.0, 0.0, 0.0)

    # Sort atoms by distance to the reference atom
    sorted_atoms = sort_atoms_by_distance(atoms, reference_atom)

    # Add a new column for distance
    atoms_with_distance = add_distance_column(sorted_atoms, reference_atom)

    # Assign atom index based on distance order
    atoms_with_index = assign_atom_index(atoms_with_distance)

    # Write the sorted XYZ file with reordered columns and updated atom index
    write_sorted_xyz_file(atoms_with_index, output_file_path)

    print(f"Sorted XYZ file with reordered columns and updated atom index written to: {output_file_path}")

Sorted XYZ file with reordered columns and updated atom index written to: centered_ice_prd.644.03.f.txt
